In [1]:
import os
import pandas as pd
import random
from sklearn.utils import shuffle
from sklearn import model_selection
import gensim
from tqdm import tqdm
import operator
import numpy as np

In [2]:
tnews_path = f"{os.getcwd()}\Desktop\机器学习\project\\news\\news.csv"
data = pd.read_csv(tnews_path)

In [3]:
data = shuffle(data)

In [4]:
data['label'].replace({'FAKE': 0, 'REAL': 1},inplace=True)

In [5]:
X = data['text']
y = data['label']

In [6]:
def read_corpus(X):
    for i, line in enumerate(X):
        tokens = gensim.utils.simple_preprocess(line)
        yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

In [7]:
X_corpus = list(read_corpus(X))

In [8]:
# X_corpus[0]

In [9]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=128, min_count=2, epochs=40)

In [10]:
model.build_vocab(X_corpus)

In [11]:
model.train(X_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [12]:
vector = model.infer_vector(X[0].split())
print(vector)

[ 2.7822216  -3.259992    0.39491537 -1.0753413   3.8570395   0.03561564
  0.53340507 -0.10072754 -0.40642005  1.4914569  -1.4142218  -1.0751823
  1.1993576  -2.3195813  -1.4040062  -1.5186428  -0.37044564 -4.8154564
 -1.9429184  -1.6987215  -1.4137243   1.5723089   0.06810404  0.11940369
 -2.001349   -0.07783099 -1.7948765  -0.21487375 -0.11521355  1.2208914
 -0.8928913   0.29473206  0.4569607  -1.1342478   1.6608936  -4.6490374
  0.9643372  -1.6555084  -0.6520668  -0.9493216  -0.47511774  1.6702405
 -2.1761138  -0.01423297 -0.63390654  1.5615858   0.07714753 -1.4895078
 -1.6798314  -1.019602   -0.97732925  0.01004239  1.9517976  -0.09590817
  0.42695782 -0.7357871   0.19527939  2.5964625  -4.105563    1.3607483
  0.3616964  -2.239433   -0.3983777  -0.18923476  2.0978334   0.5778106
  2.046764    0.2773828  -1.7091467   0.28969976  0.20577216 -2.0656414
  1.079769    0.17616105  0.29122728 -2.0004935  -0.39482367  0.40282416
 -1.5995178   2.046415    1.4113663   0.78799385  5.3028817 

In [13]:
x_vectors = list()
for x in tqdm(X):
    x_vectors.append(model.infer_vector(x.split()))

100%|██████████████████████████████████████████████████████████████████████████████| 6335/6335 [02:31<00:00, 41.72it/s]


In [14]:
x_vectors[0]

array([ 1.87397   , -1.9400567 , -1.0905926 ,  1.2750674 ,  1.6766244 ,
       -0.02083786,  0.667582  , -0.234602  , -0.57049346, -0.45388973,
       -1.8391026 ,  1.9502839 ,  0.32645142, -3.1053321 , -2.9391525 ,
        0.42798033, -0.22437905, -2.127425  , -3.1399286 , -0.34803784,
       -1.4423984 ,  0.8606746 , -3.1637452 , -0.6106109 ,  0.3641935 ,
       -0.8556857 , -0.8797792 , -0.64079195,  0.94267535, -1.0506353 ,
        1.1377815 ,  1.1332521 , -1.4584613 , -1.3002574 ,  0.16375017,
        0.70935744, -0.36854175,  1.7138312 , -2.0790927 ,  0.7449864 ,
        0.62873244,  0.48538068, -2.8524485 , -0.5577129 , -1.1193223 ,
        4.0048747 , -0.3105476 , -1.412879  , -1.3489771 ,  1.973885  ,
        0.5356252 ,  0.05663129,  3.5387604 , -2.3190362 , -0.68551075,
        1.4751412 , -0.2303532 ,  0.80695456, -0.946799  ,  0.48254198,
       -0.4072637 ,  1.7276338 ,  0.03391769, -0.50883836, -0.3484174 ,
       -0.09390675, -0.01340707, -0.28117505,  0.42423236, -1.04

In [15]:
x = x_vectors

In [16]:
class K_means:
    def __init__(self, distance_name):
        if distance_name is None:
            self.distance_name = "euclid_distance"
        else:
            self.distance_name = distance_name

    # 计算向量k和x的距离
    def distance(self, k, x):
        if self.distance_name == "manhattan_distance":
            return self.manhattan_distance(k, x)
        elif self.distance_name == "chebyshev_distance":
            return self.chebyshev_distance(k, x)
        else:
            return self.euclid_distance(k, x)

    # 返回向量k和x的切比雪夫距离
    def chebyshev_distance(self, k, x):
        dis = []
        for i in range(len(k)):
            dis.append(abs(k[i] - x[i]))
        return max(dis)

    # 返回向量k和x的Manhattan距离
    def manhattan_distance(self, k, x):
        dis = 0
        for i in range(len(k)):
            dis += abs(k[i] - x[i])
        return dis

    # 返回向量k和x欧式距离的平分
    def euclid_distance(self, k, x):
        dis = 0
        for i in range(len(k)):
            dis += (k[i] - x[i]) ** 2
        return dis

    # 求s中所有向量的均值
    def k_average(self, s):
        average = [0] * len(x[0])
        if len(s) == 0:
            return average
        for i in range(len(s)):
            average = np.sum([average, x[s[i]]], axis=0)
        return (average / len(s)).tolist()

    # 计算x的k-means算法
    def k_x_means(self):
        # 第一步，取k=2
        k0 = x[0]
        k0_temp = [0.0] * len(k0)
        s0 = []
        k1 = x[1]
        k1_temp = [0.0] * len(k1)
        s1 = []
        counter = 0
        print(f"distance_name = {self.distance_name}")
        print(f"init \nk0={k0}\nk1={k1}")
        print("************************")
        k0 = k0.tolist()
        k1 = k1.tolist()
        while not operator.eq(k0, k0_temp) or not operator.eq(k1, k1_temp):
            for ind in range(0, len(x)):
                if self.distance(k0, x[ind]) < self.distance(k1, x[ind]):
                    s0.append(ind)
                else:
                    s1.append(ind)
            # 更新k0和k1
            counter += 1
            k0_temp = k0
            k1_temp = k1
            k0 = self.k_average(s0)
            k1 = self.k_average(s1)
            print(f"counter={counter}\ns0={s0}\ns1={s1}\nk0={k0}\nk1={k1}")
            print("-------------------------------------------")
            self.class0 = s0
            self.class1 = s1
            s0 = []
            s1 = []
        print("k-mean end!")

In [24]:
# distance = None
distance = "chebyshev_distance"  # chebyshev_distance, manhattan_distance
k_means = K_means(distance_name=distance)
k_means.k_x_means()

distance_name = manhattan_distance
init 
k0=[ 1.87397    -1.9400567  -1.0905926   1.2750674   1.6766244  -0.02083786
  0.667582   -0.234602   -0.57049346 -0.45388973 -1.8391026   1.9502839
  0.32645142 -3.1053321  -2.9391525   0.42798033 -0.22437905 -2.127425
 -3.1399286  -0.34803784 -1.4423984   0.8606746  -3.1637452  -0.6106109
  0.3641935  -0.8556857  -0.8797792  -0.64079195  0.94267535 -1.0506353
  1.1377815   1.1332521  -1.4584613  -1.3002574   0.16375017  0.70935744
 -0.36854175  1.7138312  -2.0790927   0.7449864   0.62873244  0.48538068
 -2.8524485  -0.5577129  -1.1193223   4.0048747  -0.3105476  -1.412879
 -1.3489771   1.973885    0.5356252   0.05663129  3.5387604  -2.3190362
 -0.68551075  1.4751412  -0.2303532   0.80695456 -0.946799    0.48254198
 -0.4072637   1.7276338   0.03391769 -0.50883836 -0.3484174  -0.09390675
 -0.01340707 -0.28117505  0.42423236 -1.0462027   0.07912897  3.2240863
 -1.6328495   0.7669192   0.5199452   0.61179936 -1.9051319  -0.49338382
  0.81174576  1.

counter=2
s0=[0, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 30, 32, 33, 34, 35, 36, 37, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 71, 72, 73, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 90, 92, 93, 94, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 110, 111, 112, 113, 114, 115, 116, 117, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 137, 138, 140, 141, 142, 144, 145, 146, 148, 149, 150, 151, 152, 154, 155, 156, 157, 158, 159, 160, 162, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 179, 180, 182, 183, 184, 185, 186, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 203, 204, 205, 206, 207, 208, 209, 210, 212, 213, 214, 215, 216, 217, 219, 220, 222, 223, 224, 225, 226, 227, 228, 229, 231, 232, 233, 234, 235, 236, 237, 238, 240, 241, 242, 243, 245, 248, 249, 250, 25

counter=3
s0=[0, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 30, 32, 33, 34, 35, 36, 39, 41, 42, 43, 44, 45, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 59, 60, 62, 63, 64, 65, 66, 67, 68, 69, 72, 73, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 88, 92, 93, 94, 96, 97, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 110, 111, 112, 113, 114, 115, 116, 117, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 137, 138, 140, 141, 142, 144, 145, 146, 148, 149, 150, 151, 152, 154, 155, 159, 160, 162, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 179, 180, 182, 183, 184, 185, 188, 189, 190, 191, 192, 193, 194, 196, 197, 198, 199, 200, 201, 203, 204, 205, 206, 207, 208, 209, 210, 212, 213, 214, 215, 216, 217, 219, 222, 224, 225, 226, 228, 229, 231, 233, 234, 235, 236, 237, 238, 240, 241, 242, 243, 245, 248, 249, 251, 253, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270

counter=4
s0=[0, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 30, 32, 33, 34, 35, 36, 39, 41, 42, 43, 44, 45, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 59, 62, 63, 64, 65, 66, 67, 68, 69, 72, 73, 75, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 88, 92, 93, 94, 96, 97, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 110, 111, 112, 113, 114, 115, 116, 117, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 134, 135, 137, 138, 140, 142, 144, 145, 146, 148, 149, 150, 151, 152, 155, 159, 160, 162, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 179, 180, 182, 183, 184, 185, 188, 189, 190, 191, 192, 193, 194, 196, 197, 198, 199, 200, 201, 203, 204, 205, 206, 207, 208, 210, 212, 213, 214, 215, 216, 217, 219, 224, 226, 228, 231, 233, 234, 235, 236, 237, 238, 240, 241, 242, 243, 244, 245, 248, 249, 251, 253, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 266, 267, 268, 269, 270, 274, 275, 276, 278, 280, 281, 283, 284, 2

counter=5
s0=[0, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 26, 27, 28, 30, 32, 33, 34, 35, 36, 39, 41, 42, 43, 44, 45, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 59, 62, 63, 64, 65, 66, 67, 68, 69, 72, 73, 75, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 88, 92, 93, 94, 96, 97, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 110, 111, 112, 113, 114, 115, 116, 117, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 134, 135, 137, 138, 140, 142, 144, 145, 146, 148, 149, 150, 151, 152, 155, 159, 160, 161, 162, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 176, 177, 179, 180, 182, 183, 184, 185, 188, 189, 190, 191, 192, 193, 194, 196, 197, 198, 199, 201, 203, 204, 205, 206, 207, 208, 210, 212, 213, 214, 215, 216, 217, 224, 228, 231, 232, 233, 234, 235, 236, 237, 238, 240, 241, 242, 243, 244, 245, 248, 249, 251, 253, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 266, 268, 269, 270, 274, 275, 276, 278, 280, 281, 283, 284, 285, 286, 287, 

...

counter=42
s0=[0, 1, 4, 5, 7, 8, 9, 10, 11, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 28, 29, 32, 33, 34, 35, 36, 39, 41, 42, 43, 44, 45, 47, 48, 49, 50, 51, 52, 53, 54, 55, 57, 58, 59, 61, 62, 64, 66, 67, 68, 70, 71, 72, 73, 75, 80, 82, 84, 85, 86, 91, 92, 93, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 108, 110, 111, 112, 114, 115, 116, 117, 120, 121, 122, 123, 124, 128, 129, 130, 134, 135, 137, 138, 142, 143, 144, 146, 149, 150, 152, 154, 158, 159, 160, 161, 162, 164, 165, 167, 170, 171, 172, 179, 180, 182, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 196, 197, 198, 199, 201, 203, 204, 205, 206, 207, 210, 211, 212, 213, 214, 215, 217, 228, 234, 235, 236, 237, 238, 240, 241, 242, 243, 244, 245, 249, 251, 254, 255, 256, 259, 260, 263, 264, 267, 268, 269, 270, 273, 275, 276, 280, 281, 283, 284, 286, 289, 290, 293, 294, 295, 296, 297, 298, 299, 301, 302, 303, 308, 309, 310, 311, 312, 313, 314, 318, 319, 322, 323, 324, 326, 327, 330, 331, 332, 333, 334, 335, 337, 3

counter=43
s0=[0, 1, 4, 5, 7, 8, 9, 10, 11, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 28, 29, 32, 33, 34, 35, 36, 39, 41, 42, 43, 44, 45, 47, 48, 49, 50, 51, 52, 53, 54, 55, 57, 58, 59, 61, 62, 64, 66, 67, 68, 70, 71, 72, 73, 75, 80, 82, 84, 85, 86, 91, 92, 93, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 108, 110, 111, 112, 114, 115, 116, 117, 120, 121, 122, 123, 124, 128, 129, 130, 134, 135, 137, 138, 142, 143, 144, 146, 149, 150, 152, 154, 158, 159, 160, 161, 162, 164, 165, 167, 170, 171, 172, 179, 180, 182, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 196, 197, 198, 199, 201, 203, 204, 205, 206, 207, 210, 211, 212, 213, 214, 215, 217, 228, 234, 235, 236, 237, 238, 240, 241, 242, 243, 244, 245, 249, 251, 254, 255, 256, 259, 260, 263, 264, 267, 268, 269, 270, 273, 275, 276, 280, 281, 283, 284, 286, 289, 290, 293, 294, 295, 296, 297, 298, 299, 301, 302, 303, 308, 309, 310, 311, 312, 313, 314, 318, 319, 322, 323, 324, 326, 327, 330, 331, 332, 333, 334, 335, 337, 3

counter=44
s0=[0, 1, 4, 5, 7, 8, 9, 10, 11, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 28, 29, 32, 33, 34, 35, 36, 39, 41, 42, 43, 44, 45, 47, 48, 49, 50, 51, 52, 53, 54, 55, 57, 58, 59, 61, 62, 64, 66, 67, 68, 70, 71, 72, 73, 75, 80, 82, 84, 85, 86, 91, 92, 93, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 108, 110, 111, 112, 114, 115, 116, 117, 120, 121, 122, 123, 124, 128, 129, 130, 134, 135, 137, 138, 142, 143, 144, 146, 149, 150, 152, 154, 158, 159, 160, 161, 162, 164, 165, 167, 170, 171, 172, 179, 180, 182, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 196, 197, 198, 199, 201, 203, 204, 205, 206, 207, 210, 211, 212, 213, 214, 215, 217, 228, 234, 235, 236, 237, 238, 240, 241, 242, 243, 244, 245, 249, 251, 254, 255, 256, 259, 260, 263, 264, 267, 268, 269, 270, 273, 275, 276, 280, 281, 283, 284, 286, 289, 290, 293, 294, 295, 296, 297, 298, 299, 301, 302, 303, 308, 309, 310, 311, 312, 313, 314, 318, 319, 322, 323, 324, 326, 327, 330, 331, 332, 333, 334, 335, 337, 3

counter=45
s0=[0, 1, 4, 5, 7, 8, 9, 10, 11, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 28, 29, 32, 33, 34, 35, 36, 39, 41, 42, 43, 44, 45, 47, 48, 49, 50, 51, 52, 53, 54, 55, 57, 58, 59, 61, 62, 64, 66, 67, 68, 70, 71, 72, 73, 75, 80, 82, 84, 85, 86, 91, 92, 93, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 108, 110, 111, 112, 114, 115, 116, 117, 120, 121, 122, 123, 124, 128, 129, 130, 134, 135, 137, 138, 142, 143, 144, 146, 149, 150, 152, 154, 158, 159, 160, 161, 162, 164, 165, 167, 170, 171, 172, 179, 180, 182, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 196, 197, 198, 199, 201, 203, 204, 205, 206, 207, 210, 211, 212, 213, 214, 215, 217, 228, 234, 235, 236, 237, 238, 240, 241, 242, 243, 244, 245, 249, 251, 254, 255, 256, 259, 260, 263, 264, 267, 268, 269, 270, 273, 275, 276, 280, 281, 283, 284, 286, 289, 290, 293, 294, 295, 296, 297, 298, 299, 301, 302, 303, 308, 309, 310, 311, 312, 313, 314, 318, 319, 322, 323, 324, 326, 327, 330, 331, 332, 333, 334, 335, 337, 3

counter=46
s0=[0, 1, 4, 5, 7, 8, 9, 10, 11, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 28, 29, 32, 33, 34, 35, 36, 39, 41, 42, 43, 44, 45, 47, 48, 49, 50, 51, 52, 53, 54, 55, 57, 58, 59, 61, 62, 64, 66, 67, 68, 70, 71, 72, 73, 75, 80, 82, 84, 85, 86, 91, 92, 93, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 108, 110, 111, 112, 114, 115, 116, 117, 120, 121, 122, 123, 124, 128, 129, 130, 134, 135, 137, 138, 142, 143, 144, 146, 149, 150, 152, 154, 158, 159, 160, 161, 162, 164, 165, 167, 170, 171, 172, 179, 180, 182, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 196, 197, 198, 199, 201, 203, 204, 205, 206, 207, 210, 211, 212, 213, 214, 215, 217, 228, 234, 235, 236, 237, 238, 240, 241, 242, 243, 244, 245, 249, 251, 254, 255, 256, 259, 260, 263, 264, 267, 268, 269, 270, 273, 275, 276, 280, 281, 283, 284, 286, 289, 290, 293, 294, 295, 296, 297, 298, 299, 301, 302, 303, 308, 309, 310, 311, 312, 313, 314, 318, 319, 322, 323, 324, 326, 327, 330, 331, 332, 333, 334, 335, 337, 3

In [18]:
s0 = k_means.class0
s1 = k_means.class1
y = y.to_list()
print(len(s0),len(s1),len(s0)+len(s1))
print(len(y))

3809 2526 6335
6335


In [19]:
y1 = [0]*(len(s0)+len(s1))
y2 = [0]*(len(s0)+len(s1))
for ind in s0:
    y1[ind] = 1
for ind in s1:
    y2[ind] = 1

In [20]:
print(y1[:10])
print(y2[:10])
print(y[:10])

[1, 1, 1, 0, 1, 1, 0, 1, 1, 1]
[0, 0, 0, 1, 0, 0, 1, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [21]:
from sklearn.metrics import classification_report

In [22]:
y1_eval =  '\ny1_eval\n' + classification_report(y,y1,digits=6)
print(y1_eval)


y1_eval
              precision    recall  f1-score   support

           0   0.594220  0.474399  0.527592      3164
           1   0.563402  0.676758  0.614900      3171

    accuracy                       0.575691      6335
   macro avg   0.578811  0.575579  0.571246      6335
weighted avg   0.578794  0.575691  0.571294      6335



In [23]:
y2_eval =  '\ny2_eval\n' + classification_report(y,y2,digits=6)
print(y2_eval)


y2_eval
              precision    recall  f1-score   support

           0   0.436598  0.525601  0.476983      3164
           1   0.405780  0.323242  0.359839      3171

    accuracy                       0.424309      6335
   macro avg   0.421189  0.424421  0.418411      6335
weighted avg   0.421172  0.424309  0.418346      6335

